In [1]:
import json
import numpy as np
import pandas as pd
import pickle
import torch
MOVIES_FILENAME = '../../auxiliary/movies.pkl'
FOOD_FILENAME = '../../auxiliary/food.pkl'
ANN_TRAIN = '../../auxiliary/raw_data/annotated_en_train.txt'
ANN_DEV = '../../auxiliary/raw_data/annotated_en_dev.txt'
ANN_TEST='../../auxiliary/raw_data/annotated_en_test.txt'

SAVE_TRAIN = '../data/processed/durecdial2_food_train_placeholder'
SAVE_DEV = '../data/processed/durecdial2_food_dev_placeholder'
SAVE_TEST= '../data/processed/durecdial2_food_test_placeholder'

In [2]:
movies_file = open(MOVIES_FILENAME, 'rb')
movies = pickle.load(movies_file)

food_file = open(FOOD_FILENAME, 'rb')
food = pickle.load(food_file) 

In [3]:
movie_id = dict()
id_movie = dict()
for i in range(len(movies)):
    movie_id[movies[i]] = i+1
    id_movie[i+1] = movies[i]



food_id = dict()
id_food = dict()
for i in range(len(food)):
    food_id[food[i]] = i+1
    id_food[i+1] = food[i]

In [19]:
def process_data(filename, entity_list, entity_id, id_entity):
    file = open(filename, 'rb')
    lines = file.readlines()
    # point = json.loads(lines[0])
    # print(point['conversation'])
    conversations = []
    for line in lines:
        point = json.loads(line)
        utterances = point['conversation']
        current = []
        for i in range(len(utterances)):
            utt, seen_entities = processed_utterance(utterances[i], entity_list, entity_id, id_entity)
            utt, seen_entities = further_process(utt, seen_entities, entity_id)
            if i%2==0:
                utt = 'B: ' + utt
            else:
                utt = 'A: ' + utt
            # if utt.count('@') > 1:
            #     print(f'REPLACING    {utt}')
            #     print(f'Seen Entity   {seen_entities}')
            # # print(utt)
            current.append((utt,seen_entities))
        
        conversations.append(current)

    return conversations

def further_process(utterance, entities, entity_id):
    words = utterance.split(' ')
    new_utterance = ''
    for word in words:
        if '@' in word:
            new_utterance += '[MOVIE_ID] '
        else:
            new_utterance += word + ' '

    new_utterance = new_utterance.strip()
    if entities is None:
        return new_utterance,None

    int_entities = []
    for ent in entities:
        int_entities.append(entity_id[ent])
    
    

    return new_utterance,int_entities

def processed_utterance(utterance, entities, entity_id, id_entity):
    seen_entities = []
    for entity in entities:
        if entity in utterance:
            utterance = utterance.replace(entity, f'@{entity_id[entity]}')            
    words = utterance.split(' ')
    for word in words:
        if len(word)>0 and word[0]=='@':
            idx = 4
            for i in range(1, len(word)):
                if word[i].isnumeric()==False:
                    idx = i 
                    break
            seen_entities.append(id_entity[int(word[1:idx])])
            
                 
    if seen_entities==[]:
        return (utterance, None)

    return (utterance, seen_entities)


In [20]:
processed_train  = process_data(ANN_TRAIN, food, food_id, id_food)
torch.save(processed_train,SAVE_TRAIN)

In [23]:
processed_dev  = process_data(ANN_DEV, food, food_id, id_food)
torch.save(processed_dev,SAVE_DEV)

In [24]:
processed_test  = process_data(ANN_TEST, food, food_id, id_food)
torch.save(processed_test,SAVE_TEST)

In [25]:
id_movie[1]

"Dragon's Love"

In [26]:
food_dictionary = {}
for id in id_food.keys():
    food_dictionary[id] = id_food[id]

In [27]:
torch.save(food_dictionary,'../data/processed/durecdial2_full_food_db_placeholder')